# 基于GPT-2的"关键词生成广告文案"任务（样例测试demo）
> 在GPT-2上做了5个epoch的fine-tune

## 1. 导入依赖

In [2]:
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import torch.nn as nn
import numpy as np
import pandas as pd

## 2. 设置device，拿到tokenizer，实例化带CausalLM head的model

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("gpt2-chinese-cluecorpussmall")
model = AutoModelForCausalLM.from_pretrained("gpt2-chinese-cluecorpussmall").to(device)
model.eval()
torch.load('model/gpt2_gen_ads.pth', map_location=torch.device('cpu'))
print('model loaded')

model loaded


## 3. 定义生成算法

In [21]:
def gen_ads(input_text, max_length=260):
    token = tokenizer(input_text, return_tensors='pt')
    input_ids = token['input_ids']
    attention_mask = token['attention_mask']

    with torch.no_grad():
        output_ids = model.generate(input_ids=input_ids, attention_mask=attention_mask, do_sample=True, num_beams=1, max_length=max_length, min_length=1, no_repeat_ngram_size=3, early_stopping=True)
        output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return output_text

## 4. 样例测试

In [22]:
input_text = '类型#裤*风格#复古*风格#简约*风格#休闲*图案#字母*图案#文字*图案#复古*图案#线条*图案#撞色*裤型#直筒裤*裤款式#抽褶'

output_text = gen_ads(input_text)
output_text = output_text.replace(' ', '')
output_text = output_text[len(input_text):]
output_text



Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


'*裤子面料休闲版型设计，不是简单的大码运动裤，为小巧的腿部带来全新的个性效果，采用大面积拉链，方便用户穿脱，方能更好的突出腿型，展现出大个性，更加的时尚前卫感。大方、大方的廓型设置，彰显得腰线自然和线条轮廓。同时，宽松有型的款式也会凸显出腿部线条，大方中不失小性感！牛仔衬衫，不仅美观而且实用，上身拉裤子是可以让人时尚休闲的选择，上紧下宽的紧身牛仔裤的版型，大度且直，不会太过于单调，'